# Download the data from : https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

# Classifying IMDB Sentiments with BERT Tokenizer and Tensorflow 2.0

In [1]:
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
import pandas as pd
import numpy as np
import re
import math

# Tensorflow 2.0

In [31]:
print(tf.__version__)

2.0.0


In [2]:
df = pd.read_csv("IMDB Dataset.csv")

In [3]:
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [4]:
TAG_RE = re.compile(r'<[^>]+>')
def clean_text(sentence):

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    sentence = TAG_RE.sub('', sentence)

    return sentence

In [5]:
df['clean_reviews'] = df['review'].astype(str).apply(clean_text)

In [6]:
df

,review,sentiment,clean_reviews
0,One of the other reviewers has mentioned that ...,positive,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,A wonderful little production br br The filmin...
2,I thought this was a wonderful way to spend ti...,positive,I thought this was wonderful way to spend time...
3,Basically there's a family where a little boy ...,negative,Basically there a family where little boy Jake...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,Petter Mattei Love in the Time of Money is vis...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,I thought this movie did down right good job I...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,Bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,negative,I am Catholic taught in parochial elementary s...
49998,I'm going to have to disagree with the previou...,negative,I going to have to disagree with the previous ...


In [7]:
y = df['sentiment']

y = df['sentiment'].map({'positive':1,'negative':0})

In [8]:
print(df['clean_reviews'][10])

Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines br br At first it was very odd and pretty funny but as the movie progressed didn find the jokes or oddness funny anymore br br Its low budget film thats never problem in itself there were some pretty interesting characters but eventually just lost interest br br imagine this film would appeal to stoner who is currently partaking br br For something similar but better try Brother from another planet 


In [9]:
print(y[10])

0


In [10]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [11]:
tokenizer.tokenize("Salman Brother is an actor for the masses")

['salman', 'brother', 'is', 'an', 'actor', 'for', 'the', 'masses']

In [12]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

[2123, 2102, 2022, 2061, 8689, 2389]

In [13]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [14]:
tokenized_reviews = [tokenize_reviews(review) for review in df.review]

In [15]:
reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]

In [16]:
import random

In [17]:
random.shuffle(reviews_with_len)

In [18]:
reviews_with_len.sort(key=lambda x: x[2])

In [19]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [20]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [21]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [22]:
next(iter(batched_dataset))

(<tf.Tensor: id=23054, shape=(32, 28), dtype=int32, numpy=
 array([[ 3191,  1996,  2338,  1010,  5293,  1996,  3185,   999,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [ 3078,  5436,   999,  3078,  3257,   999,  3532,  7613,  1012,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [ 2023,  3185,  2003,  6659,  2021,  2009,  2038,  2070,  2204,
          3896,  1012,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [ 2054,  1037,  5896,  1010,  2054,  1037,  2466,  1010,  2054,
          1037,  6752,   999,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [

In [23]:
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [24]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [25]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [26]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [27]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="nadam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="nadam",
                       metrics=["sparse_categorical_accuracy"])

In [28]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
1407/1407 [==============================] - 493s 350ms/step - loss: 0.2946 - accuracy: 0.8692
Epoch 2/5
1407/1407 [==============================] - 496s 352ms/step - loss: 0.1343 - accuracy: 0.9511
Epoch 3/5
1407/1407 [==============================] - 473s 336ms/step - loss: 0.0521 - accuracy: 0.9824
Epoch 4/5
1407/1407 [==============================] - 478s 340ms/step - loss: 0.0266 - accuracy: 0.9910
Epoch 5/5
1407/1407 [==============================] - 475s 338ms/step - loss: 0.0214 - accuracy: 0.9922


In [30]:
%timeit
results = text_model.evaluate(test_dataset)
print(results)

NameError: name 'test_dataset' is not defined